In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as web
import matplotlib.pyplot as plt

from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
from pandas import DataFrame

In [2]:
#   #
def d1(S,K,T,r,sigma):
    return(log(S/K) + (r+sigma**2/2.)*T)/(sigma*sqrt(T))
    

In [3]:
print(d1(50,40,0.25,0.3,0.2))

3.031435513142098


In [4]:
def d2(S,K,T,r,sigma):
    return (d1(S,K,T,r,sigma) - sigma*sqrt(T))

In [5]:
print(d2(50,40,0.25,0.3,0.2))

2.9314355131420977


In [6]:
def black_call(S,K,T,r,sigma):
    return (S*norm.cdf(d1(S,K,T,r,sigma)) - K*exp(-r*T) * norm.cdf(d2(S,K,T,r,sigma)))

In [7]:
print(black_call(50,40,0.25,0.3,0.2))

12.892016091242283


In [8]:
def black_put(S,K,T,r,sigma):
    return(K*exp(-r*T)-S+black_call(S,K,T,r,sigma))

In [9]:
print(black_call(50,40,0.25,0.3,0.2))

12.892016091242283


In [10]:
stock = 'SPY'
#stock é a opção de compra#
expiry = '12-30-2021' # Mes - dia - ano# 
# dia que expira a  opção #
strike_price = 365
#preço exercido de 370 #
today = datetime.now()
#pega o dia de hoje#
one_year = today.replace(year=today.year-1)
#pega o dia anterior e o ano aterior #

In [11]:
df = web.DataReader(stock, 'yahoo', one_year,today)

In [12]:
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-10-07,341.630005,338.089996,338.119995,340.760010,56999600.0,336.026581
2020-10-08,343.850006,341.859985,342.850006,343.779999,45242500.0,339.004578
2020-10-09,347.350006,344.890015,345.559998,346.850006,59528600.0,342.031952
2020-10-12,354.019989,349.059998,349.589996,352.429993,80388500.0,347.534454
2020-10-13,352.470001,349.089996,352.279999,350.130005,73255500.0,345.266418


In [13]:
df = df.sort_values(by="Date")
df = df.dropna()
df = df.assign(close_day_before=df.Close.shift(1))
df['return'] = ((df.Close - df.close_day_before )/df.close_day_before)

In [14]:
df.head()

,High,Low,Open,Close,Volume,Adj Close,close_day_before,return
Date,,,,,,,,
2020-10-07,341.630005,338.089996,338.119995,340.760010,56999600.0,336.026581,NaN,NaN
2020-10-08,343.850006,341.859985,342.850006,343.779999,45242500.0,339.004578,340.760010,0.008863
2020-10-09,347.350006,344.890015,345.559998,346.850006,59528600.0,342.031952,343.779999,0.008930
2020-10-12,354.019989,349.059998,349.589996,352.429993,80388500.0,347.534454,346.850006,0.016088
2020-10-13,352.470001,349.089996,352.279999,350.130005,73255500.0,345.266418,352.429993,-0.006526


In [15]:
# volatividade calculada #
sigma = np.sqrt(252) * df['return'].std()

In [16]:
uty = (web.DataReader("^TNX", 'yahoo', today.replace(day=today.day-1), today)['Close'].iloc[-1])/100
uty

0.015640000104904173

In [17]:
# pegando o ultimo valor de Close SPY #
lcp = df['Close'].iloc[-1]
lcp

440.69000244140625

In [18]:
T = (datetime.strptime(expiry, "%m-%d-%Y") - datetime.utcnow()).days/365
if T < 0:
    T = T*(-1) 

In [19]:
print("A opção de preço é:", black_call(lcp,strike_price, T, uty, sigma))

A opção de preço é: 76.99727044888817


In [20]:
def call_Volatility(Price, S,K,T,r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = S * \
            norm.cdf(d1(S, K, T, r, sigma))-K*exp(-r*T) * \
            norm.cdf(d2(S, K, T, r, sigma))
        if Price - (Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

In [21]:
print("Call Volatilidade : " +str(100 * call_Volatility(black_call(lcp, strike_price, T, uty, sigma,), lcp, strike_price, T, uty,)) + " %")

Call Volatilidade : 13.500000000000009 %


In [22]:
def put_Volatility(Price, S, K, T,r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = K*exp(-r*T)-S+black_call(S, K, T, r, sigma)
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

In [23]:
print("Put Volatilidade : " +str(100 * put_Volatility(black_call(lcp, strike_price, T, uty, sigma,), lcp, strike_price, T, uty,)) + " %")

Put Volatilidade : Not FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot FoundNot Found %


In [24]:
df

,High,Low,Open,Close,Volume,Adj Close,close_day_before,return
Date,,,,,,,,
2020-10-07,341.630005,338.089996,338.119995,340.760010,56999600.0,336.026581,NaN,NaN
2020-10-08,343.850006,341.859985,342.850006,343.779999,45242500.0,339.004578,340.760010,0.008863
2020-10-09,347.350006,344.890015,345.559998,346.850006,59528600.0,342.031952,343.779999,0.008930
2020-10-12,354.019989,349.059998,349.589996,352.429993,80388500.0,347.534454,346.850006,0.016088
2020-10-13,352.470001,349.089996,352.279999,350.130005,73255500.0,345.266418,352.429993,-0.006526
...,...,...,...,...,...,...,...,...
2021-10-01,436.029999,427.230011,430.980011,434.239990,129240100.0,434.239990,429.140015,0.011884
2021-10-04,433.959991,426.359985,433.000000,428.640015,128570000.0,428.640015,434.239990,-0.012896
2021-10-05,435.489990,429.390015,430.239990,433.100006,90682500.0,433.100006,428.640015,0.010405


In [25]:
def d1(S,K,T,r,q,sigma):
    return(log(S/K)+(r - q +sigma**2/2.)*T)/sigma*sqrt(T)

In [26]:
def d2(S,K,T,r,q,sigma):
    return d1(S,K,T,r,q,sigma)-sigma*sqrt(T)

In [27]:
def bs_call(S,K,T,r,q,sigma):
    return S*exp(-q*T)*norm.cdf(d1(S,K,T,r,q,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,q,sigma))

In [28]:
def bs_put(S,K,T,r,q,sigma):
    return K*exp(-r*T)-S*exp(-q*T)+bs_call(S,K,T,r,q,sigma)

In [29]:
def Bissec_Volatility(op, option_price, S, K, r, T, q):
    # apply bisection method to get the implied volatility by solving the BSM function
    precision = 0.00001
    upper_vol = 50.0
    max_vol = 50.0
    min_vol = 0.0001
    lower_vol = 0.0001
    iteration = 0    

    while 1:
        iteration +=1 
        mid_vol = (upper_vol + lower_vol)/2.0
        
        if op == 'c':
            price = bs_call(S,K,T,r,q,mid_vol)
            lower_price = bs_call(S,K,T,r,q,lower_vol)
            if (lower_price - option_price) * (price - option_price) > 0:
                lower_vol = mid_vol 
            else:
                upper_vol = mid_vol
            if abs(price - option_price) < precision: break
            if mid_vol > max_vol - 5 : 
                mid_vol = 0.0001
                break

        elif op == 'p':
            price = bs_put(S,K,T,r,q,mid_vol)
            upper_price = bs_put(S,K,T,r,q,upper_vol)

            if (upper_price - option_price) * (price - option_price) > 0:
                upper_vol = mid_vol 
            else:
                lower_vol = mid_vol            
                      
            if abs(price - option_price) < precision: break 
            
            if iteration > 100: break
                
    return mid_vol 

In [30]:
print(Bissec_Volatility('c',5.57,100,100,0.05,1,0))

0.061087350559234616


In [31]:
print(Bissec_Volatility('p',5.57,100,100,0.05,1,0))

0.199906290602684
